In [1]:
import pyspark
from delta import *

In [2]:
builder = (pyspark.sql.SparkSession.builder.appName("Alter-Column")
           .config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension")
           .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
           )

In [3]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/vk/Library/Python/3.9/lib/python/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/vk/.ivy2/cache
The jars for the packages stored in: /Users/vk/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-457c6f2c-dc6e-4ca4-9643-63ab2311ed65;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 88ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   

23/04/07 20:16:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
columns = ["Name","Country"]
data = [("ABC","USA"),("QAZ","India"),("WSX","Singapore")]
rdd=spark.sparkContext.parallelize(data)
df=rdd.toDF(columns)

In [5]:
df.show()

+----+---------+
|Name|  Country|
+----+---------+
| ABC|      USA|
| QAZ|    India|
| WSX|Singapore|
+----+---------+



In [11]:
spark.sql("DROP TABLE IF EXISTS MY_DELTALAKE_TABLE")

DataFrame[]

In [12]:
df.write.format("delta").saveAsTable("my_deltalake_table")

23/04/07 20:24:35 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [13]:
spark.sql("select * from my_deltalake_table").show()

+----+---------+
|Name|  Country|
+----+---------+
| WSX|Singapore|
| QAZ|    India|
| ABC|      USA|
+----+---------+



In [14]:
DeltaTable.isDeltaTable(spark,"spark-warehouse/my_deltalake_table/")

True

In [15]:
spark.sql("DESCRIBE HISTORY my_deltalake_table").show()

+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|           operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      0|2023-04-07 20:24:...|  null|    null|CREATE TABLE AS S...|{isManaged -> tru...|null|    null|     null|       null|  Serializable|         true|{numFiles -> 4, n...|        null|Apache-Spark/3.3....|
+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+-----------

In [18]:
spark.sql("""ALTER TABLE `my_deltalake_table` SET TBLPROPERTIES (
          'delta.columnMapping.mode'='name',
          'delta.minReaderVersion'='2',
          'delta.minWriterVersion'='5')"""
          )

DataFrame[]

In [19]:
spark.sql("ALTER TABLE `my_deltalake_table` RENAME TO `whatever`")

DataFrame[]

In [20]:
DeltaTable.isDeltaTable(spark,"spark-warehouse/whatever/")

True

In [23]:
spark.sql("DESCRIBE HISTORY whatever").select("version","timestamp").show(truncate=False)

+-------+-----------------------+
|version|timestamp              |
+-------+-----------------------+
|1      |2023-04-07 20:33:38.94 |
|0      |2023-04-07 20:24:34.986|
+-------+-----------------------+



In [24]:
spark.sql("ALTER TABLE whatever RENAME COLUMN Name TO ID")

DataFrame[]

In [25]:
spark.sql("select * from whatever").show()

+---+---------+
| ID|  Country|
+---+---------+
|WSX|Singapore|
|QAZ|    India|
|ABC|      USA|
+---+---------+



In [31]:
spark.table('whatever').columns

['ID', 'Country']

In [32]:
spark.sql("ALTER TABLE WHATEVER DROP COLUMN COUNTRY")

DataFrame[]

In [33]:
spark.table('whatever').columns

['ID']